<p style="font-size: 18px;">
  This is the accompanying code for the post titled "Game-Changing Algorithms: SEDS in Python for Better Decision-Making"<br>
  You can find it <a href="https://pureai.substack.com/p/game-changing-algorithms">here</a>.<br>
  Published: September 9, 2023<br>
  <a href="https://pureai.substack.com">https://pureai.substack.com</a>
</p>

Welcome to this Jupyter notebook! If you're new to Python or don't have it installed on your system, don't worry; you can still follow along and explore the code.

Here's a quick guide to getting started:

- Using an Online Platform: You can run this notebook in a web browser using platforms like Google Colab or Binder. These services offer free access to Jupyter notebooks and don't require any installation.
- Installing Python Locally: If you'd prefer to run this notebook on your own machine, you'll need to install Python. A popular distribution for scientific computing is Anaconda, which includes Python, Jupyter, and other useful tools.
  - Download Anaconda from [here](https://www.anaconda.com/download).
  - Follow the installation instructions for your operating system.
  - Launch Jupyter Notebook from Anaconda Navigator or by typing jupyter notebook in your command line or terminal.
- Opening the Notebook: Once you have Jupyter running, navigate to the location of this notebook file (.ipynb) and click on it to open.
- Running the Code: You can run each cell in the notebook by selecting it and pressing Shift + Enter. Feel free to modify the code and experiment with it.
- Need More Help?: If you're new to Python or Jupyter notebooks, you might find these resources helpful:
  - [Python.org's Beginner's Guide](https://docs.python.org/3/tutorial/index.html)
  - [Jupyter Notebook Basics](https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Notebook%20Basics.html)

Happy coding, and enjoy exploring the fascinating world of Adversarial Search Algorithms!

In [1]:
# A small readme
from platform import python_version
print(f"The version of Python used is: {python_version()}")

import random
from typing import List, Tuple, Dict, Callable, Set, Optional

The version of Python used is: 3.10.12


_The following Jupyter notebook is an adaptation of a programming assignment in Johns Hopkins' Artificial Intelligence course, part of the Whiting School of Engineering EP Master's program._

## Solving Normal Form Games

In the lecture we talked about the Prisoner's Dilemma game, shown here in Normal Form:

Player 1 / Player 2  | Defect | Cooperate
------------- | ------------- | -------------
Defect  | -5, -5 | -1, -10
Cooperate  | -10, -1 | -2, -2

where the payoff to Player 1 is the left number and the payoff to Player 2 is the right number. We can represent each payoff cell as a Tuple: `(-5, -5)`, for example. We can represent each row as a List of Tuples: `[(-5, -5), (-1, -10)]` would be the first row and the entire table as a List of Lists:

In [2]:
prisoners_dilemma = [
 [( -5, -5), (-1,-10)],
 [(-10, -1), (-2, -2)]]

prisoners_dilemma

[[(-5, -5), (-1, -10)], [(-10, -1), (-2, -2)]]

in which case the strategies are represented by indices into the List of Lists. For example, `(Defect, Cooperate)` for the above game becomes `prisoners_dilemma[ 0][ 1]` and returns the payoff `(-1, -10)` because 0 is the first row of the table ("Defect" for Player 1) and 1 is the 2nd column of the row ("Cooperate" for Player 2).

For this assignment, you are going write a function that uses Successive Elimination of Dominated Strategies (SEDS) to find the **pure strategy** Nash Equilibrium of a Normal Form Game. The function is called `solve_game`:

```python
def solve_game( game: List[List[Tuple]], weak=False) -> Tuple:
    pass # returns strategy indices of Nash equilibrium or None.
```

and it takes two parameters: the game, in a format that we described earlier and an optional boolean flag that controls whether the algorithm considers only **strongly dominated strategies** (the default will be false) or whether it should consider **weakly dominated strategies** as well.

It should work with game matrices of any size and it will return the **strategy indices** of the Nash Equilibrium. If there is no **pure strategy** equilibrium that can be found using SEDS, return `None`.


<div style="background: mistyrose; color: firebrick; border: 2px solid darkred; padding: 5px; margin: 10px;">
Do not return the payoff. That's not useful. Return the strategy indices, any other output is incorrect.
</div>

As before, you must provide your implementation in the space below, one Markdown cell for documentation and one Code cell for implementation, one function and assertations per Codecell.


---

### eliminate_strategies_player1

*This function takes in a game matrix, the remaining rows and cols and determines if there are strongly or weakly dominated strategies for Player 1. If there are, it removes them from the game matrix set and returns the eliminated boolean.*

* **game** List[List[Tuple[int,int]]]: The game matrix
* **remaining_rows** Set[int]: The remaining rows for Player 1
* **remaining_cols** Set[int]: The remaining cols for Player 2
* **weak** bool: Whether to consider weakly dominated strategies

*Returns: eliminated bool: Whether any strategies were eliminated*

In [3]:
def eliminate_strategies_player1(game: List[List[Tuple[int, int]]], remaining_rows: Set[int], remaining_cols: Set[int], weak: bool = False) -> bool:
    eliminated = False
    for i in list(remaining_rows):
        for j in list(remaining_rows):
            if i != j:
                dominates = all(game[j][k][0] >= game[i][k][0] for k in remaining_cols) if weak else all(game[j][k][0] > game[i][k][0] for k in remaining_cols)
                if dominates:
                    remaining_rows.remove(i)
                    eliminated = True
                    break
    return eliminated

### unit test: eliminate_strategies_player1

In [4]:
# Arrange
# Example game matrix
prisoners_dilemma = [
    [(-5, -5), (-1, -10)],
    [(-10, -1), (-2, -2)]
]

# Set of indices for the remaining strategies for player 1 and 2
remaining_rows = set(range(len(prisoners_dilemma)))
assert remaining_rows == {0, 1}, "Expected remaining strategies to be {0, 1}"
remaining_cols = set(range(len(prisoners_dilemma[0])))

# Act
eliminated = eliminate_strategies_player1(prisoners_dilemma, remaining_rows, remaining_cols)

# Assert the function has eliminated a strategy for player 1
assert eliminated == True

# Assert that remaining_rows has changed (meaning a strategy was eliminated)
assert remaining_rows == {0}, "Expected remaining strategies to be {0}"

### eliminate_strategies_player2

*This function takes in a game matrix, the remaining rows and cols and determines if there are strongly or weakly dominated strategies for Player 2. If there are, it removes them from the game matrix set and returns the eliminated boolean.*

* **game** List[List[Tuple[int,int]]]: The game matrix
* **remaining_rows** Set[int]: The remaining rows for Player 1
* **remaining_cols** Set[int]: The remaining cols for Player 2
* **weak** bool: Whether to consider weakly dominated strategies

*Returns: eliminated bool: Whether any strategies were eliminated*

In [5]:
def eliminate_strategies_player2(game: List[List[Tuple[int, int]]], remaining_rows: Set[int], remaining_cols: Set[int], weak: bool = False) -> bool:
    eliminated = False
    for k in list(remaining_cols):
        for l in list(remaining_cols):
            if k != l:
                dominates = all(game[i][l][1] >= game[i][k][1] for i in remaining_rows) if weak else all(game[i][l][1] > game[i][k][1] for i in remaining_rows)
                if dominates:
                    remaining_cols.remove(k)
                    eliminated = True
                    break
    return eliminated

### unit test: eliminate_strategies_player2

In [6]:
# Arrange
# Example game matrix
prisoners_dilemma = [
    [(-5, -5), (-1, -10)],
    [(-10, -1), (-2, -2)]
]

# Set of indices for the remaining strategies for player 1 and 2
remaining_rows = set(range(len(prisoners_dilemma)))
remaining_cols = set(range(len(prisoners_dilemma[0])))
assert remaining_cols == {0, 1}, "Expected remaining strategies to be {0, 1}"

# Act
eliminated = eliminate_strategies_player2(prisoners_dilemma, remaining_rows, remaining_cols)

# Assert the function has eliminated a strategy for player 2
assert eliminated == True

# Assert that remaining_rows has changed (meaning a strategy was eliminated)
assert remaining_cols == {0}, "Expected remaining strategies to be {0}"

### eliminate_dominated_strategies

*This function takes in a game matrix, the remaining rows and cols and determines and a player and then calls for each player to determine if they have any strongly dominant strategies. If there are, it removes them from the game matrix set and returns the eliminated boolean.*

* **game** List[List[Tuple[int,int]]]: The game matrix
* **remaining_rows** Set[int]: The remaining rows for Player 1
* **remaining_cols** Set[int]: The remaining cols for Player 2
* **weak** bool: Whether to consider weakly dominated strategies
* **player** int: The player (1 or 2) to check for dominant strategies

*Returns: eliminated bool: Whether any strategies were eliminated*

In [7]:
def eliminate_dominated_strategies(game: List[List[Tuple[int, int]]], remaining_rows: Set[int], remaining_cols: Set[int], weak: bool, start_player: int) -> bool:
    eliminated = False
    
    # Choose the order based on the starting player
    if start_player == 1:
        eliminated = eliminate_strategies_player1(game, remaining_rows, remaining_cols, weak)
        eliminated = eliminate_strategies_player2(game, remaining_rows, remaining_cols, weak)
    else:
        eliminated = eliminate_strategies_player2(game, remaining_rows, remaining_cols, weak)
        eliminated = eliminate_strategies_player1(game, remaining_rows, remaining_cols, weak)
    
    return eliminated

### unit test: eliminate_dominated_strategies

In [8]:
# Example game matrix
test_game = [[(10, 10), (14, 12), (14, 15)], 
             [(12, 14), (20, 20), (28, 15)],
             [(15, 14), (15, 28), (25, 25)]]

# Test with player 1 starting
remaining_rows = set(range(len(test_game)))
remaining_cols = set(range(len(test_game[0])))

# Call the function and store the result
eliminated = eliminate_dominated_strategies(test_game, remaining_rows, remaining_cols, weak=False, start_player=1)

# Assert the function has eliminated a strategy for player 1 and/or player 2
assert eliminated == True, "Expected True, since some strategies are dominated"

# Assert that remaining_rows and remaining_cols have changed (meaning a strategy was eliminated)
assert remaining_rows == {1, 2}, "Expected remaining strategies for player 1 to be {0, 2} when player 2 starts"
assert remaining_cols == {1}, "Expected remaining strategies for player 2 to be {1} when player 2 starts"

### solve_game

*Solves the specified game using the SEDS algorithm. Depending upon which player starts, each player will attempt to eliminate their weaker strategies by looking for either strongly dominant strategies or weakly dominant strategies. Player 1 operates on the strategies in the rows and Player 2 operates on the strategies in the columns. If a pure strategy Nash equilibrium exists, the remaining rows and remaining cols will be one each, which points to the indices of the equilibrium point. If no Nash Equilibrium point exists, the function returns None.*

* **game** List[List[Tuple[int, int]]]: The game matrix where each element is a tuple (payoff_player_0, payoff_player_1)
* **weak** bool: If True, look for weakly dominated strategies. If False, look for strictly dominated strategies.

**returns** Tuple: The indices of the Nash equilibrium strategies, or None if no such strategies exist.

In [9]:
def solve_game(game: List[List[Tuple[int, int]]], weak: bool = False) -> Tuple[int, int]:
    # Pick a random start player between 1 and 2
    start_player = random.randint(1, 2)

    remaining_rows = set(range(len(game)))
    remaining_cols = set(range(len(game[0])))
    
    while True:
        # Eliminate dominated strategies
        eliminated = eliminate_dominated_strategies(game, remaining_rows, remaining_cols, weak, start_player)
        # If no strategy was eliminated in this iteration, we are done
        if not eliminated:
            break
    
    # Return remaining strategies as indices
    if len(remaining_rows) == 1 and len(remaining_cols) == 1:
        return [(i, j) for i in remaining_rows for j in remaining_cols][0]
    else:
        None

### unit tests: solve_game

In [10]:
# Example usage
prisoners_dilemma = [
    [(-5, -5), (-1, -10)],
    [(-10, -1), (-2, -2)]
]

# Solving the game using Successive Elimination of Dominated Strategies (SEDS)
assert (solve_game(prisoners_dilemma) == (0,0))

test_game = [[(20, 20), (28, 15)], 
             [(15, 28), (25, 25)]]

assert (solve_game(test_game) == (0,0))

self_check_example = [[(10, 10), (14, 12), (14, 15)], 
                      [(12, 14), (20, 20), (28, 15)],
                      [(15, 14), (15, 28), (25, 25)]]

assert (solve_game(self_check_example) == (1,1))

## Additional Directions

Create three games as described and according to the following:

1. Your games must be created and solved "by hand".
2. The strategy pairs must **not** be on the main diagonal (0, 0), (1, 1), or (2, 2). And the solution cannot be the same for both Game 1 and Game 2.
3. Make sure you fill out the Markdown ("?") with your game as well as the solution ("?").
4. Remember, **do not return the payoff**, return the strategy indices.

For games that can be solved with *weak* SEDS, there may be more than one solution. You only need to return the first solution found. However, if you would like to return all solutions, you can implement `solve_game` as state space search.

### Test Game 1. Create a 3x3 two player game

**that can only be solved using the Successive Elimintation of Strongly Dominated Strategies**

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | (6,4) | (1,0) | (3,2) |
|1  | (8,7) | (4,3) | (5,3) |
|2  | (4,3) | (2,1) | (4,3) |

**Solution:** (1,0)

The steps to reproduce this game are (if player 1 starts):

We check for strongly dominant strategies for player 1 (a comparison of the first value in the tuple between the rows). We see that row 1 is strictly dominant over row 0, so we remove row 0 from the game matrix. 8 > 6 && 4 > 1 && 5 > 3.

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | - | - | - |
|1  | (8,7) | (4,3) | (5,3) |
|2  | (4,3) | (2,1) | (4,3) |

Player 2 now has a turn to check for strongly dominant strategies (a comparison of the second value in the tuple between the columns). We see that column 0 is strictly dominant over column 1, so we remove column 1 from the game matrix. That is 7 > 3 && 3 > 1.

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | - | - | - |
|1  | (8,7) | - | (5,3) |
|2  | (4,3) | - | (4,3) |

For Player 1's turn, we look for strongly dominant strategies. We see that row 1 is strictly dominant over row 2, so we remove row 2 from the game matrix. 8 > 4 && 5 > 4.

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | - | - | - |
|1  | (8,7) | - | (5,3) |
|2  | - | - | - |

Lastly, Player 2 has a chance to look for strongly dominant strategies. We find that 7 > 3, thus column 0 is strictly dominant over column 2, so we remove column 2 from the game matrix.

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | - | - | - |
|1  | (8,7) | - | - |
|2  | - | - | - |

We end with the solution being (1,0)

In [11]:
test_game_1 = [[(6, 4), (1, 0), (3, 2)], 
               [(8, 7), (4, 3), (5, 3)],
               [(4, 3), (2, 1), (4, 3)]]

solution = solve_game(test_game_1)
print(solution)

(1, 0)


In [12]:
assert solution == (1,0)

### Test Game 2. Create a 3x3 two player game

**that can only be solved using the Successive Elimintation of Weakly Dominated Strategies**

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | (5,3) | (1,1) | (3,3) |
|1  | (2,0) | (3,3) | (0,0) |
|2  | (5,3) | (4,0) | (3,1) |

**Solution:** (2,0)

In [13]:
test_game_2 = [[(5, 3), (1, 1), (3, 3)], 
               [(2, 0), (3, 3), (0, 0)],
               [(5, 3), (4, 0), (3, 1)]]

strong_solution = solve_game(test_game_2)
weak_solution = solve_game(test_game_2, weak=True)

In [14]:
assert strong_solution == None
assert weak_solution == (2,0)

### Test Game 3. Create a 3x3 two player game

**that cannot be solved using the Successive Elimintation of Dominated Strategies at all**

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | (0,3) | (0,1) | (0,0) |
|1  | (2,0) | (12,12) | (1,6) |
|2  | (4,3) | (2,0) | (3,1) |

**Solution:** None

In [15]:
test_game_3 = [[(0, 3), (0, 1), (0, 0)], 
               [(2, 0), (12, 12), (1, 6)],
               [(4, 3), (2, 0), (3, 1)]]

strong_solution = solve_game(test_game_3)
weak_solution = solve_game(test_game_3, weak=True)

In [16]:
assert strong_solution == None
assert weak_solution == None